# Prepare

**import**

In [17]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import sys
sys.path.append("./util_")
import acquire
import prepare

### get data

In [16]:
# get data from acquire file
dogs = acquire.get_data()
dogs.head()

,UniqueID,DateOfBite,Species,Breed,Age,Gender,SpayNeuter,Borough,ZipCode
0,1,January 01 2018,DOG,UNKNOWN,NaN,U,False,Brooklyn,11220
1,2,January 04 2018,DOG,UNKNOWN,NaN,U,False,Brooklyn,NaN
2,3,January 06 2018,DOG,Pit Bull,NaN,U,False,Brooklyn,11224
3,4,January 08 2018,DOG,Mixed/Other,4,M,False,Brooklyn,11231
4,5,January 09 2018,DOG,Pit Bull,NaN,U,False,Brooklyn,11224


## Remove columns

remove ZipCode beacause most of my data has NaN cells.

In [21]:
# drop redundent columns
remove_cols = ["UniqueID", "ZipCode"]
dogs = prepare.drop_cols(dogs, remove_cols)

Original dataframe size: (22663, 9)
New dataframe size: (22663, 8)


## Check column data types

In [22]:
dogs.select_dtypes("object")

,DateOfBite,Species,Breed,Age,Gender,Borough,ZipCode
0,January 01 2018,DOG,UNKNOWN,NaN,U,Brooklyn,11220
1,January 04 2018,DOG,UNKNOWN,NaN,U,Brooklyn,NaN
2,January 06 2018,DOG,Pit Bull,NaN,U,Brooklyn,11224
3,January 08 2018,DOG,Mixed/Other,4,M,Brooklyn,11231
4,January 09 2018,DOG,Pit Bull,NaN,U,Brooklyn,11224
...,...,...,...,...,...,...,...
22658,December 24 2017,DOG,CHIWEENIE MIX,7,M,Staten Island,10303
22659,December 24 2017,DOG,DUNKER,5,F,Staten Island,10303
22660,December 21 2017,DOG,"Schnauzer, Miniature",10M,M,Staten Island,10312
22661,December 28 2017,DOG,Mixed/Other,NaN,F,Staten Island,10308


In [30]:
# convert string to datetime
pd.to_datetime(dogs['DateOfBite'])

# convert string to integer
dogs.ZipCode.astype("int")

ValueError: cannot convert float NaN to integer

In [33]:
dogs.ZipCode.value_counts(dropna=False)

NaN      5858
10029     369
11208     261
11368     226
10065     221
         ... 
21040       1
12533       1
13502       1
20701       1
?           1
Name: ZipCode, Length: 520, dtype: int64